# Exercises Repetition

In [21]:
import pandas as pd # Data manipulation
from scipy.special import comb # Combinations
import matplotlib.pyplot as plt # Data visualization
from mlxtend.frequent_patterns import apriori # Apriori algorithm
from mlxtend.frequent_patterns import fpgrowth # FP-growth algorithm
from mlxtend.frequent_patterns import association_rules # Association rules

## Theoretical questions

### Question 1:
The following probabilities are known from a dataset from the previous year. 79% of students are taking the Data Science exam for the first time. Respectively `12.65%`, `4.30%` and `4.05%` of the students participate in the exam for the `2nd`, `3rd` and `4th` time. If the student participates for the first time, the chance that he or she will pass is '60%'. For the second, third and fourth participation, the chance of passing is `40%`, `12%` and `2%` respectively.

- 1.1. There is a `53.057%` chance that a randomly selected student from last year has passed. Write down the calculation that gives you this percentage based on the data from the exercise.

- 1.2.`24%` of the students who sat for the Data Science exam last year were baptized. The percentage of successful students for the group of baptized students is exactly the same as the percentage of successful students for the entire group. Suppose the teacher chooses one student from the entire group of students (who took the exam last year). What are the chances that this student was baptized and passed Data Science last year.

- 1.3. The teacher calls on `15` students who participated for the first time last year the exam. What is the probability that 10 or more students have passed?

### Question 2:
The average study time per course is `4.5` hours per week in the course. We want to determine whether the study time of the course differs significantly from the average for the Data Science course (based on a sample). Use the input file for this

- 2.1. What alternative hypothesis can we use?

- 2.2 What is the acceptance interval for the study time of the course at `a=0.05`? Also write down how you arrived at this number.

- 2.3. What value for the factor did you use to calculate the acceptance interval?

- 2.4. What can you say about the null hypothesis?

- 2.5. Calculate the p-value that you can use to support this claim (at `a=0.05`)

- 2.6. Form a conclusion based on the p-value. Complete the following sentence (in any case, use the obtained p-value in your conclusion): If the average study duration of the population is `4.5 HOURS`, then

### Question 3:
We want to know whether the going out behavior of our students differs from those of the University From a survey of all students of the
university shows the following distribution:

|                   | Behavior College | Measured education absolutely |
|-------------------|------------------|-------------------------------|
| Never             | 5.00%            | 23                            |
| Occasionally      | 30.00%           | 103                           |
| Weekly            | 35.00%           | 130                           |
| More times a week | 20.00%           | 86                            |
| Daily             | 10.00%           | 53                            |


3.1. Calculate x^2

3.2. What is the p value for this distribution?

3.3. I want to make a decision with reliability of `90%`. Which decision do I take?

### Question 4:
A store has analyzed a number of shopping carts. Look for the association rules.

4.1. Read the groceries.csv file.

In [22]:
groceries = pd.read_csv('../Data/Groceries.csv', delimiter=',', decimal='.')

In [23]:
display(groceries.head())

,transaction,items
0,1,{citrus fruit;semi-finished bread;margarine;re...
1,2,{tropical fruit;yogurt;coffee}
2,3,{whole milk}
3,4,{pip fruit;yogurt;cream cheese ;meat spreads}
4,5,{other vegetables;whole milk;condensed milk;lo...


4.2. To convert the items in string format to an n-dimensional array you can do the following perform conversion.
```python
transactions = groceries['items'].apply(lambda row: row[1:-1].split(';')).to_numpy();

# For each list of items in form {item 1, item 2, item 3}
#1. Remove the curly braces with [1:-1]
#2. Split based on “;”
#3. Convert the panda Series to a numpy ndarray
```

In [24]:
transactions = groceries['items'].apply(lambda row: row[1:-1].split(';')).to_numpy();

In [26]:
print(transactions)

[list(['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'])
 list(['tropical fruit', 'yogurt', 'coffee']) list(['whole milk']) ...
 list(['chicken', 'citrus fruit', 'other vegetables', 'butter', 'yogurt', 'frozen dessert', 'domestic eggs', 'rolls/buns', 'rum', 'cling film/bags'])
 list(['semi-finished bread', 'bottled water', 'soda', 'bottled beer'])
 list(['chicken', 'tropical fruit', 'other vegetables', 'vinegar', 'shopping bags'])]


4.3. Create the association rules with the apriori algorithm based on the following parameters:
- minimum support = `0.05`
- minimum confidence = `0.1`
- minimum lengte = `2`
- maximum lengte = `5`


4.4. Look up the line yogurt → whole milk. Is this a good line? Rate based on Support, Confidence and Lift.